In [1]:
!pip install pandas transformers


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
# Importing necessary libraries
import json
import pandas as pd
import random
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch


In [3]:
import pandas as pd

def read_csv_to_dataframe(file_path):
    """
    Reads a CSV file and converts it into a pandas DataFrame.
    
    Parameters:
    file_path (str): Path to the CSV file to be read.

    Returns:
    pandas.DataFrame: DataFrame containing the data from the CSV file.
    """
    data = pd.read_csv(file_path)
    return pd.DataFrame(data)
def replace_names(df, names_list):
    """
    Replaces placeholders with random names from the names_list in the DataFrame.

    Parameters:
    df (pandas.DataFrame): DataFrame where names need to be replaced.
    names_list (list): List of names to be used for replacement.

    Returns:
    pandas.DataFrame: Updated DataFrame with names replaced.
    """
    for index, row in df.iterrows():
        # Randomly select two names
        name1, name2 = random.sample(names_list, 2)

        # Replace placeholders in dialogue and summary
        df.at[index, 'dialogue'] = row['dialogue'].replace('#Person1#', name1).replace('#Person2#', name2)
        df.at[index, 'summary'] = row['summary'].replace('#Person1#', name1).replace('#Person2#', name2)
        
    return df

# List of names to be used for replacement
names_list = [
    'Alice', 'Bob', 'Charlie', 'Diana', 'Edward', 'Fiona', 'George', 'Hannah', 'Ian', 'Julia', 'Kevin', 'Laura',
    'Megan', 'Nathan', 'Olivia', 'Peter', 'Quincy', 'Rachel', 'Samuel', 'Tina', 'Umar', 'Violet', 'William', 'Xena',
    'Yasmin', 'Zachary', 'Amelia', 'Brian', 'Carmen', 'David', 'Elena', 'Frank', 'Grace', 'Henry', 'Isla', 'Jack',
    'Kara', 'Leo', 'Maya', 'Nolan', 'Ophelia', 'Pablo', 'Queenie', 'Raj', 'Sara', 'Tom', 'Ursula', 'Victor', 'Wendy',
    'Xander', 'Yolanda', 'Zane', 'Anita', 'Blake', 'Claire', 'Derek', 'Eve', 'Felix', 'Giselle', 'Harold', 'Ivy', 'Jasper', 'Kylie', 'Liam', 'Monica', 'Nigel', 'Opal', 'Preston', 'Quinn',
    'Rosa', 'Sebastian', 'Tracy', 'Ulysses', 'Valerie', 'Winston', 'Xiomara', 'Yvette', 'Zelda', 'Aaron', 'Brianna',
    'Cody', 'Danielle', 'Ethan', 'Farrah', 'Gavin', 'Hazel', 'Isaac', 'Jocelyn', 'Kyle', 'Luna', 'Miles', 'Nadia',
    'Orlando', 'Penelope', 'Quincy', 'Rebecca', 'Shane', 'Tara', 'Ursula', 'Vance', 'Whitney', 'Xavier', 'Yasmine',
    'Zach', 'Aurora', 'Brandon', 'Celeste'
]

In [4]:
# Path to JSONL file
train_file_path = '/workspace/bestdataset/train (1).csv'

# Reading the JSONL file and creating a DataFrame
train_df = read_csv_to_dataframe(train_file_path)
train_df = replace_names(train_df, names_list)

# Displaying the first few rows of the DataFrame with replaced names
train_df

,id,dialogue,summary,topic
0,train_0,"Nadia: Hi, Mr. Smith. I'm Doctor Hawkins. Why ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"Tina: Hello Mrs. Parker, how have you been?\nA...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"Nadia: Excuse me, did you see a set of keys?\n...",Nadia's looking for a set of keys and asks for...,find keys
3,train_3,Jasper: Why didn't you tell me you had a girlf...,Jasper's angry because Danielle didn't tell Ja...,have a girlfriend
4,train_4,"George: Watsup, ladies! Y'll looking'fine toni...",Malik invites Nikki to dance. Nikki agrees if ...,dance
...,...,...,...,...
12455,train_12455,Whitney: Excuse me. You are Mr. Green from Man...,Tan Ling picks Mr. Green up who is easily reco...,pick up someone
12456,train_12456,Hannah: Mister Ewing said we should show up at...,Hannah and Blake plan to take the underground ...,conference center
12457,train_12457,Nadia: How can I help you today?\nOphelia: I w...,Ophelia rents a small car for 5 days with the ...,rent a car
12458,train_12458,Quincy: You look a bit unhappy today. What's u...,Kara's mom lost her job. Kara hopes mom won't ...,job losing


In [5]:
import pandas as pd

# 1. Видаляємо рядки з NaN або None у колонках dialogue або summary
train_df = train_df.dropna(subset=['dialogue', 'summary'])

# 2. Видаляємо рядки, де в діалозі є ( XXXXXXXXXX )
train_df = train_df[~train_df['dialogue'].str.contains(r'\( XXXXXXXXXX \)')]

In [6]:
train_df

,id,dialogue,summary,topic
0,train_0,"Nadia: Hi, Mr. Smith. I'm Doctor Hawkins. Why ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"Tina: Hello Mrs. Parker, how have you been?\nA...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"Nadia: Excuse me, did you see a set of keys?\n...",Nadia's looking for a set of keys and asks for...,find keys
3,train_3,Jasper: Why didn't you tell me you had a girlf...,Jasper's angry because Danielle didn't tell Ja...,have a girlfriend
4,train_4,"George: Watsup, ladies! Y'll looking'fine toni...",Malik invites Nikki to dance. Nikki agrees if ...,dance
...,...,...,...,...
12455,train_12455,Whitney: Excuse me. You are Mr. Green from Man...,Tan Ling picks Mr. Green up who is easily reco...,pick up someone
12456,train_12456,Hannah: Mister Ewing said we should show up at...,Hannah and Blake plan to take the underground ...,conference center
12457,train_12457,Nadia: How can I help you today?\nOphelia: I w...,Ophelia rents a small car for 5 days with the ...,rent a car
12458,train_12458,Quincy: You look a bit unhappy today. What's u...,Kara's mom lost her job. Kara hopes mom won't ...,job losing


In [7]:
train_df = train_df.drop(columns=['id'])
train_df = train_df.drop(columns=['topic'])


In [8]:
train_df

,dialogue,summary
0,"Nadia: Hi, Mr. Smith. I'm Doctor Hawkins. Why ...","Mr. Smith's getting a check-up, and Doctor Haw..."
1,"Tina: Hello Mrs. Parker, how have you been?\nA...",Mrs Parker takes Ricky for his vaccines. Dr. P...
2,"Nadia: Excuse me, did you see a set of keys?\n...",Nadia's looking for a set of keys and asks for...
3,Jasper: Why didn't you tell me you had a girlf...,Jasper's angry because Danielle didn't tell Ja...
4,"George: Watsup, ladies! Y'll looking'fine toni...",Malik invites Nikki to dance. Nikki agrees if ...
...,...,...
12455,Whitney: Excuse me. You are Mr. Green from Man...,Tan Ling picks Mr. Green up who is easily reco...
12456,Hannah: Mister Ewing said we should show up at...,Hannah and Blake plan to take the underground ...
12457,Nadia: How can I help you today?\nOphelia: I w...,Ophelia rents a small car for 5 days with the ...
12458,Quincy: You look a bit unhappy today. What's u...,Kara's mom lost her job. Kara hopes mom won't ...


In [9]:
# Path to JSONL file
val_df_file_path = '/workspace/bestdataset/validation.csv'

# Reading the JSONL file and creating a DataFrame
val_df = read_csv_to_dataframe(val_df_file_path)

val_df = replace_names(val_df, names_list)

# Displaying the first few rows of the DataFrame with replaced names
val_df

,id,dialogue,summary,topic
0,dev_0,"Ursula: Hello, how are you doing today?\nUrsul...",Ursula has trouble breathing. The doctor asks ...,see a doctor
1,dev_1,Diana: Hey Jimmy. Let's go workout later today...,Diana invites Jimmy to go workout and persuade...,do exercise
2,dev_2,Ophelia: I need to stop eating such unhealthy ...,"Ophelia plans to stop eating unhealthy foods, ...",healthy foods
3,dev_3,Yasmine: Do you believe in UFOs?\nViolet: Of c...,Violet believes in UFOs and can see them in dr...,UFOs and aliens
4,dev_4,Isaac: Did you go to school today?\nEdward: Of...,Isaac didn't go to school today. Edward wants ...,go to school
...,...,...,...,...
495,dev_495,"Henry: Now that it's the new year, I've decide...",Henry decides to stop smoking and come out of ...,the new year
496,dev_496,"Derek: You married Joe, didn't you? \nMaya: Jo...",Derek thought Maya married Joe. Maya denies.,fall in love
497,dev_497,Peter: How can I help you mam?\nYolanda: I was...,Yolanda's car makes noises. Peter thinks it ne...,noises
498,dev_498,"Vance: Hello, Amazon's customer service. How c...",Tom calls Amazon's customer service because on...,a missing page


In [10]:
val_df = val_df.drop(columns=['id'])
val_df = val_df.drop(columns=['topic'])


In [11]:
val_df

,dialogue,summary
0,"Ursula: Hello, how are you doing today?\nUrsul...",Ursula has trouble breathing. The doctor asks ...
1,Diana: Hey Jimmy. Let's go workout later today...,Diana invites Jimmy to go workout and persuade...
2,Ophelia: I need to stop eating such unhealthy ...,"Ophelia plans to stop eating unhealthy foods, ..."
3,Yasmine: Do you believe in UFOs?\nViolet: Of c...,Violet believes in UFOs and can see them in dr...
4,Isaac: Did you go to school today?\nEdward: Of...,Isaac didn't go to school today. Edward wants ...
...,...,...
495,"Henry: Now that it's the new year, I've decide...",Henry decides to stop smoking and come out of ...
496,"Derek: You married Joe, didn't you? \nMaya: Jo...",Derek thought Maya married Joe. Maya denies.
497,Peter: How can I help you mam?\nYolanda: I was...,Yolanda's car makes noises. Peter thinks it ne...
498,"Vance: Hello, Amazon's customer service. How c...",Tom calls Amazon's customer service because on...


In [12]:
# Path to JSONL file
test_df_file_path = '/workspace/bestdataset/test (2).csv'

# Reading the JSONL file and creating a DataFrame
test_df = read_csv_to_dataframe(test_df_file_path)

test_df = replace_names(test_df, names_list)

# Displaying the first few rows of the DataFrame with replaced names
test_df

,id,dialogue,summary,topic
0,test_0_1,"Jack: Ms. Dawson, I need you to take a dictati...",Ms. Dawson helps Jack to write a memo to infor...,communication method
1,test_0_2,"Samuel: Ms. Dawson, I need you to take a dicta...",In order to prevent employees from wasting tim...,company policy
2,test_0_3,"Sebastian: Ms. Dawson, I need you to take a di...",Ms. Dawson takes a dictation for Sebastian abo...,dictation
3,test_1_1,Julia: You're finally here! What took so long?...,Opal arrives late because of traffic jam. Juli...,public transportation
4,test_1_2,Megan: You're finally here! What took so long?...,Ivy decides to follow Megan's suggestions on q...,transportation
...,...,...,...,...
1495,test_498_2,Zach: Matthew? Hi!\nValerie: Steve! Haven't se...,Matthew and Steve meet after a long time. Stev...,finding a house
1496,test_498_3,Megan: Matthew? Hi!\nPenelope: Steve! Haven't ...,Steve has been looking for a place to live. Ma...,find a house
1497,test_499_1,"Olivia: Hey, Betsy, did you hear the great new...",Frank invites Besty to the party to celebrate ...,party invitation
1498,test_499_2,"Victor: Hey, Betsy, did you hear the great new...",Frank invites Betsy to the big promotion party...,promotion party invitation


In [13]:
test_df = test_df.drop(columns=['id'])
test_df = test_df.drop(columns=['topic'])

In [14]:
test_df

,dialogue,summary
0,"Jack: Ms. Dawson, I need you to take a dictati...",Ms. Dawson helps Jack to write a memo to infor...
1,"Samuel: Ms. Dawson, I need you to take a dicta...",In order to prevent employees from wasting tim...
2,"Sebastian: Ms. Dawson, I need you to take a di...",Ms. Dawson takes a dictation for Sebastian abo...
3,Julia: You're finally here! What took so long?...,Opal arrives late because of traffic jam. Juli...
4,Megan: You're finally here! What took so long?...,Ivy decides to follow Megan's suggestions on q...
...,...,...
1495,Zach: Matthew? Hi!\nValerie: Steve! Haven't se...,Matthew and Steve meet after a long time. Stev...
1496,Megan: Matthew? Hi!\nPenelope: Steve! Haven't ...,Steve has been looking for a place to live. Ma...
1497,"Olivia: Hey, Betsy, did you hear the great new...",Frank invites Besty to the party to celebrate ...
1498,"Victor: Hey, Betsy, did you hear the great new...",Frank invites Betsy to the big promotion party...


In [15]:
# Loading data
train = pd.read_csv('/workspace/bestdataset2/samsum-train.csv')
test = pd.read_csv('/workspace/bestdataset2/samsum-test.csv')
val = pd.read_csv('/workspace/bestdataset2/samsum-validation.csv')

In [16]:
import re

def clean_tags(text):
    clean = re.compile('<.*?>') # Compiling tags
    clean = re.sub(clean, '', text) # Replacing tags text by an empty string

    # Removing empty dialogues
    clean = '\n'.join([line for line in clean.split('\n') if not re.match('.*:\s*$', line)])

    return clean

In [17]:
test1 = clean_tags(train['dialogue'].iloc[2145]) # Applying function to example text
test2 = clean_tags(test['dialogue'].iloc[0]) # Applying function to example text



print(test2)

Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye


In [18]:
# Defining function to clean every text in the dataset.
def clean_df(df, cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(clean_tags)
    return df

In [19]:
# Cleaning texts in all datasets
train = clean_df(train,['dialogue', 'summary'])
test = clean_df(test,['dialogue', 'summary'])
val = clean_df(val,['dialogue', 'summary'])

In [20]:
import re

def remove_emojis(text: str) -> str:
    if text is None:
        return ""
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # смайлики
        u"\U0001F300-\U0001F5FF"  # символи та піктограми
        u"\U0001F680-\U0001F6FF"  # транспорт і карти
        u"\U0001F1E0-\U0001F1FF"  # прапори
        u"\U00002700-\U000027BF"  # різні символи
        u"\U0001F900-\U0001F9FF"  # додаткові смайлики
        u"\U00002600-\U000026FF"  # сонце, зірки, інше
        "]+", 
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

# застосовуємо до всіх частин датасету
for df in [train, val, test]:
    df["dialogue"] = df["dialogue"].apply(remove_emojis)
    df["summary"] = df["summary"].apply(remove_emojis)


In [21]:
import re

def remove_text_smileys(text: str) -> str:
    if text is None:
        return ""
    # стандартні смайли
    smiley_pattern = r'[:;=8][\-~]?[)DPOp3\(\]/\\]'
    # серце <3 та інші подібні символи
    text = re.sub(smiley_pattern, '', text)
    text = re.sub(r'<3', '', text)
    return text


# застосовуємо до всіх датасетів
for df in [train, val, test]:
    df["dialogue"] = df["dialogue"].apply(remove_text_smileys)
    df["summary"] = df["summary"].apply(remove_text_smileys)


In [22]:
train

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,"Theresa: Hey Louise, how are u?\r\nTheresa: Th...",Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [23]:
import re

# Об’єднуємо словники скорочень
all_abbr = {
    # Твій abbreviation_map
    "u": "you",
    "ur": "your",
    "r": "are",
    "btw": "by the way",
    "asap": "as soon as possible",
    "tbh": "to be honest",
    "lol": "laughing out loud",
    "omg": "oh my god",
    "sth": "something",
    "s/o": "shout out",
    "pls": "please",
    "plz": "please",
    "FR": "France",
    "bc": "because",
    "b/c": "because",
    "w/": "with",
    "w/o": "without",
    "idk": "I don’t know",
    "imo": "in my opinion",
    "fyi": "for your information",
    "jk": "just kidding",
    "thx": "thanks",
    "tx": "thanks",
    "np": "no problem",
    "lmk": "let me know",
    # Твій chat_words словник
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laughter",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can’t stop laughing"
}

# Створюємо ключі у нижньому регістрі
all_abbr_lower = {k.lower(): v for k, v in all_abbr.items()}

def expand_all_abbreviations(text):
    if text is None:
        return ""
    
    def replace(match):
        word = match.group(0)
        key = word.lower()
        return all_abbr_lower.get(key, word)
    
    pattern = re.compile(r'\b\w+\/?\w*\b', flags=re.IGNORECASE)
    return pattern.sub(replace, text)

# Застосування до всіх датасетів
for df in [train, val, test]:
    df['dialogue'] = df['dialogue'].apply(expand_all_abbreviations)
    df['summary'] = df['summary'].apply(expand_all_abbreviations)


In [24]:
train

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood to be hon...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,"Theresa: Hey Louise, how are You?\r\nTheresa: ...",Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [25]:
import re

def remove_empty_dialogues(text):
    if text is None:
        return ""
    
    # Розбиваємо на рядки і залишаємо тільки ті, де є текст після ":"
    clean_lines = [line for line in text.split('\n') if re.search(r':\s*\S+', line)]
    
    return '\n'.join(clean_lines)

# Застосування до всіх датасетів
for df in [train, val, test]:
    df['dialogue'] = df['dialogue'].apply(remove_empty_dialogues)

In [26]:
# Concatenating the DataFrames
trainik_df = pd.concat([train, val, test, val_df, train_df, test_df])

# Reseting the index
trainik_df.reset_index(drop=True, inplace=True)

# Displaying the result
trainik_df

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood to be hon...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
30823,NaN,Zach: Matthew? Hi!\nValerie: Steve! Haven't se...,Matthew and Steve meet after a long time. Stev...
30824,NaN,Megan: Matthew? Hi!\nPenelope: Steve! Haven't ...,Steve has been looking for a place to live. Ma...
30825,NaN,"Olivia: Hey, Betsy, did you hear the great new...",Frank invites Besty to the party to celebrate ...
30826,NaN,"Victor: Hey, Betsy, did you hear the great new...",Frank invites Betsy to the big promotion party...


In [27]:
trainik_df = trainik_df.drop(columns=['id'])

In [28]:
trainik_df 


,dialogue,summary
0,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,"Tim: Hi, what's up?\r\nKim: Bad mood to be hon...",Kim may try the pomodoro technique recommended...
3,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...
30823,Zach: Matthew? Hi!\nValerie: Steve! Haven't se...,Matthew and Steve meet after a long time. Stev...
30824,Megan: Matthew? Hi!\nPenelope: Steve! Haven't ...,Steve has been looking for a place to live. Ma...
30825,"Olivia: Hey, Betsy, did you hear the great new...",Frank invites Besty to the party to celebrate ...
30826,"Victor: Hey, Betsy, did you hear the great new...",Frank invites Betsy to the big promotion party...


In [29]:
!pip install scikit-learn



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split



# 70% train, 30% temp
train_df, temp_df = train_test_split(trainik_df, test_size=0.30, random_state=42)

# 10% validation, 20% test із залишку
val_df, test_df = train_test_split(temp_df, test_size=2/3, random_state=42)

# Перевірка розмірів
print(f"Train: {len(train_df)}")
print(f"Validation: {len(val_df)}")
print(f"Test: {len(test_df)}")

# Збереження у файли
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)


Train: 21579
Validation: 3083
Test: 6166


In [31]:
from torch.utils.data import Dataset

class SummaryDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer  
        self.data = data            
        self.max_length = max_length 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]  
        dialogue = item['dialogue']  
        summary = item['summary']   

        # додаємо префікс до діалогу
        source_text = dialogue

        source = self.tokenizer.encode_plus(
            source_text,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )

        target = self.tokenizer.encode_plus(
            summary,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': source['input_ids'].flatten(),
            'attention_mask': source['attention_mask'].flatten(),
            'labels': target['input_ids'].flatten(),
            'summary': summary 
        }

In [32]:
!pip install sentencepiece


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [33]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Токенайзер і модель BART-base
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')


In [34]:
train_dataset = SummaryDataset(tokenizer, train_df)

valid_dataset = SummaryDataset(tokenizer, val_df)

In [35]:
!pip install evaluate rouge_score bert-score


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [36]:
def make_compute_metrics(tokenizer):
    import evaluate
    import torch
    import numpy as np

    rouge_metric = evaluate.load("rouge")
    bertscore_metric = evaluate.load("bertscore")

    def compute_metrics(eval_pred):
        preds, labels = eval_pred

        # замінюємо -100 (pad label) на tokenizer.pad_token_id
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

        # декодуємо
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # додатково міняємо None на ""
        decoded_preds = [str(p) if p is not None else "" for p in decoded_preds]
        decoded_labels = [str(l) if l is not None else "" for l in decoded_labels]

        rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        rouge_scores = {key: value * 100 for key, value in rouge_result.items()}

        bertscore_result = bertscore_metric.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
        bertscore_f1 = torch.tensor(bertscore_result["f1"]).mean().item() * 100

        return {**rouge_scores, "bertscore_f1": bertscore_f1}

    return compute_metrics


In [37]:
!pip install transformers[torch]


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [44]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps",       # оцінка під час тренування
    eval_steps=1000,                    # частіша валідація, щоб швидше бачити покращення
    save_strategy="steps",
    save_steps=1000,
    learning_rate=3e-5,                # трохи менше LR може дати стабільніший тренінг
    per_device_train_batch_size=16,     # якщо GPU пам’яті вистачає, можна збільшити
    per_device_eval_batch_size=16,
    num_train_epochs=30,               # 20-30 достатньо, більше ризик overfitting
    weight_decay=0.01,
    predict_with_generate=True,
    logging_steps=100,
    report_to="none",
    fp16=True,
    gradient_accumulation_steps=2,     
    save_total_limit=3,
)


In [45]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    processing_class=tokenizer,   # замість tokenizer=
    compute_metrics=make_compute_metrics(tokenizer)
)

In [46]:
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bertscore F1
1000,0.091000,0.087808,45.590137,21.896963,38.250178,38.255832,90.583134
2000,0.083800,0.085571,45.756066,22.649825,38.757907,38.748534,90.615016
3000,0.071000,0.085185,46.393831,22.836243,39.089978,39.084053,90.677369
4000,0.069800,0.084958,46.459639,23.461274,39.487926,39.492259,90.734512
5000,0.060900,0.086960,46.873236,23.479211,39.634677,39.647396,90.793127
6000,0.058100,0.088142,47.021397,23.690142,39.640647,39.651016,90.750343


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [48]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer
import torch

In [57]:
# === 1. Шлях до вашого чекпоінта ===
checkpoint_dir = "/workspace/results/checkpoint-5000"

# === 2. Завантажуємо модель і токенайзер ===
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_dir)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

In [58]:
# === 3. Нові аргументи тренера для оцінки ===
eval_args = Seq2SeqTrainingArguments(
    output_dir="./results_eval",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    fp16=True,
    do_train=False,
    do_eval=True,
    logging_steps=100,
    report_to="none"
)

In [59]:
test_dataset = SummaryDataset(tokenizer, test_df)

In [60]:
# === 4. Ініціалізація Trainer для оцінки ===
trainer = Seq2SeqTrainer(
    model=model,
    args=eval_args,
    eval_dataset=test_dataset,            # тестовий датасет
    tokenizer=tokenizer,
    compute_metrics=make_compute_metrics(tokenizer)  # ті самі метрики
)

/tmp/ipykernel_812/3197383196.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# === 5. Оцінка на тесті ===
results = trainer.evaluate()
print("Метрики на тестовому наборі:", results)
